In [1]:
import pandas as pd
import numpy as np

read_csv(  
    &emsp;file = "path",  
    &emsp;sep = "thing",  
    &emsp;header = 0, if there is no header then header = None  
    &emsp;names = ["columnname", "columnname"] - column names,  
    &emsp;index_col = "column to use as index",  
    &emsp;usecols = [list of columns to keep in df],  
    &emsp;dtype = {dictionary of column names and datatypes},  
    &emsp;parse_dates = [column to parse] - converts date string into datetimes,  
    &emsp;infer_datetime_format = True,  
    &emsp;na_values = [list of strings to recognize as NaN],  
    &emsp;nrows = int, number of rows to read in  
    &emsp;skiprows = [list of ints] line numbers to skip (accepts lambda functions)  
    &emsp;converters = {dictionary with column names and functions to apply}  
)

In [2]:
pd.read_csv("../data/product.csv",
            index_col="PRODUCT_ID",
            parse_dates=False)

UsageError: Cell magic `%%black` not found.
